# 核心功能

> 读取因子数据的核心模块

In [ ]:
#| default_exp __init__

In [ ]:
#| export
import numpy as np
import pandas as pd
import deeplake
from loguru import logger
import tqdm

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# | export
def __initialize_factor(
    fac_key: str,
    fac_df: pd.DataFrame,
    token: str,
    fac_name: str,
    fac_report: str,
    fac_wechat_link: str,
):
    """请勿使用此函数"""
    ds_object = deeplake.load("hub://chenzongwei/factor", token=token)
    ds_object.factors_list.append(
        {
            "fac_key": fac_key,
            "fac_name": fac_name,
            "fac_report": fac_report,
            "fac_wechat_link": fac_wechat_link,
        }
    )
    logger.success(f"已经将代号为{fac_key}的{fac_name}因子的相关信息写入因子信息表")
    """处理df数据的部分"""
    fac_df.index = fac_df.index.strftime("%Y%m%d").astype(int)
    fac_df.columns = fac_df.columns.str.slice(start=0, stop=-3).astype(int)
    fac_df = fac_df.stack().reset_index()
    fac_df.columns = ["date", "code", "fac"]
    dates = fac_df.iloc[:, 0]
    dates = sorted(list(set(dates)))
    """创建新的group和tensor"""
    ds_object.create_group(fac_key)
    ds_object[fac_key].create_tensor("value")
    ds_object[fac_key].create_tensor("date", htype="class_label")
    logger.success("已经成功创建新的group和tensor")
    """每天写入"""
    for date in tqdm.auto.tqdm(dates):
        son = fac_df[fac_df.date == date].to_numpy()
        ds_object[fac_key].append({"value": son, "date": date})
    logger.success(f"{dates[0]}到{dates[-1]}的数据全部写入完成了")

In [ ]:
# | export
def __update_factor(fac_key: str, fac_df: pd.DataFrame, token: str):
    """请勿使用此函数"""
    ds_object = deeplake.load("hub://chenzongwei/factor", token=token)
    """找到最近新增的部分，处理df数据的部分"""
    old_dates = [i[0] for i in ds_object[fac_key].date.numpy().tolist()]
    new_dates = [
        i
        for i in list(fac_df.index)
        if i not in [pd.Timestamp(str(j)) for j in old_dates]
    ]
    fac_df = fac_df[fac_df.index.isin(new_dates)]
    fac_df.index = fac_df.index.strftime("%Y%m%d").astype(int)
    fac_df.columns = fac_df.columns.str.slice(start=0, stop=-3).astype(int)
    fac_df = fac_df.stack().reset_index()
    fac_df.columns = ["date", "code", "fac"]
    dates = fac_df.iloc[:, 0]
    dates = sorted(list(set(dates)))
    if len(dates) > 0:
        """每天更新写入"""
        for date in tqdm.auto.tqdm(dates):
            son = fac_df[fac_df.date == date].to_numpy()
            ds_object[fac_key].append({"value": son, "date": date})
        logger.success(f"{dates[0]}到{dates[-1]}的数据全部更新完成了")
    else:
        logger.warning("已经是最新的了，无需更新")

In [ ]:
# | export
def show_all_factors_information(token: str) -> pd.DataFrame:
    """展示目前数据库内包含的所有因子数据的相关信息，包括：    
    1. 数据键名：提取数据时需要使用的数据键名，即`read_factor`函数中的第一个参数`fac_key`，如：factor1    
    2. 因子名称：研报中因子的名称，如：适度冒险因子    
    3. 报告题目：发布该因子的研究报告的题目，如：成交量激增时刻蕴含的alpha信息——多因子选股系列研究之一    
    4. 微信链接：该报告在微信公众号上的的宣传推文的链接    

    Parameters
    ----------
    token : str
        验证码

    Returns
    -------
    pd.DataFrame
        相关信息的表格
    """
    ds_object = deeplake.load("hub://chenzongwei/factor", token=token)
    keys = [i[0] for i in ds_object.factors_list.fac_key[:].data()["text"]]
    names = [i[0] for i in ds_object.factors_list.fac_name[:].data()["text"]]
    reports = [i[0] for i in ds_object.factors_list.fac_report[:].data()["text"]]
    wechat_links = [
        i[0] for i in ds_object.factors_list.fac_wechat_link[:].data()["text"]
    ]
    info = pd.DataFrame(
        {"数据键名": keys, "因子名称": names, "报告题目": reports, "微信链接": wechat_links}
    )
    return info

In [ ]:
# | export
def read_factor(
    fac_key: str,
    token: str,
    trade_date: int = None,
    start_date: int = None,
    end_date: int = None,
    sql_like: bool = False,
) -> pd.DataFrame:
    """通过表名，读取因子数据

    Parameters
    ----------
    fac_key : str
        表的名称
    token : str
        验证码
    trade_date : int, optional
        读取单日因子值，形如20230113，指定此参数时，start_date和end_date两个参数将失效, by default None
    start_date : int, optional
        读取因子值的起始日期，形如20130101, by default None
    end_date : int, optional
        读取因子值的终止日期，形如20221231, by default None
    sql_like : bool, optional
        返回的数据为形如sql中的长表，包括日期、股票代码、因子值三列, by default False

    Returns
    -------
    pd.DataFrame
        因子值，index为每天的日期，columns为股票代码，values为因子值
    """

    def wind_code(x):
        if x[0] in ["0", "3"]:
            return x + ".SZ"
        elif x[0] == "6":
            return x + ".SH"
        elif x[0] == "8":
            return x + ".BJ"
        else:
            return x + ".UN"

    ds_object = deeplake.load("hub://chenzongwei/factor", token=token)
    """找到起止日期的序号"""
    all_dates = [i[0] for i in ds_object[fac_key].date.numpy().tolist()]
    if trade_date is not None:
        try:
            trade_num = all_dates.index(trade_date)
            fac_data = ds_object[fac_key].value[trade_num].numpy()
        except Exception:
            raise ValueError(f"暂时没有{trade_date}这一天的因子值")
    else:
        if start_date is not None:
            try:
                start_date = [i for i in all_dates if i <= start_date][-1]
            except Exception:
                raise ValueError(f"暂时没有{start_date}及之后的因子值")
            start_num = all_dates.index(start_date)
        else:
            start_num = 0
        if end_date is not None:
            end_date = [i for i in all_dates if i >= end_date][0]
            fac_data = np.vstack(
                ds_object[fac_key]
                .value[start_num : (all_dates.index(end_date) + 1)]
                .numpy(aslist=True)
            )
        else:
            fac_data = np.vstack(
                ds_object[fac_key].value[start_num:].numpy(aslist=True)
            )
    """整理数据"""
    fac_data = pd.DataFrame(fac_data, columns=["date", "code", fac_key])
    fac_data.date = pd.to_datetime(
        fac_data.date.astype(int).astype(str), format="%Y%m%d"
    )
    fac_data.code = fac_data.code.astype(int).astype(str).str.zfill(6)
    fac_data = fac_data.pivot(index="date", columns="code", values=fac_key)
    fac_data.columns = [wind_code(i) for i in list(fac_data.columns)]
    if sql_like:
        fac_data = fac_data.stack().reset_index()
        fac_data.columns = ["date", "code", fac_key]
    return fac_data

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()